In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from utils.TD import TD_SGD, TD_Adam

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

seed = 42
np.random.seed(seed=seed)

In [2]:
housedata = np.loadtxt('data\\readyhousedata.txt', delimiter=',')

X = housedata[:, :-1]
y = housedata[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg = LinearRegression() 
reg.fit(X_train, y_train)

weights = reg.coef_
intercept = reg.intercept_

print("Weights learned using LR:\n", weights)
print("Intercept:", intercept)

score = reg.score(X_test, y_test)
print("Model R^2 score:", score)

Weights learned using LR:
 [-10.99366542   5.00578355   0.63581661   2.44872684 -10.31186138
  17.51763825   1.03114679 -17.77574728   7.34087816  -6.42763345
  -8.79966234   3.03563353 -20.51935608]
Intercept: 29.04431567479631
Model R^2 score: 0.7730570034661727


In [3]:
def mini_batch_sgd(
        X: np.ndarray, 
        y: np.ndarray, 
        learning_rate: float, 
        n_iter: int, 
        batch_size: int,
        epsilon: float = 0, 
    ) -> np.ndarray:
    """
    Performs Mini-Batch Stochastic Gradient Descent (SGD) for linear regression with shuffling.
    
    Parameters:
    X (np.ndarray): Feature matrix (n_samples, n_features).
    y (np.ndarray): Target vector (n_samples,).
    learning_rate (float): Step size for updating weights.
    n_iter (int): Number of iterations (epochs).
    epsilon (float): Convergence threshold.
    batch_size (int): Size of the mini-batches.
    
    Returns:
    np.ndarray: The learned weights.
    """
    n_samples, n_features = X.shape
    # Add bias term to the feature matrix
    X_bias = np.c_[np.ones(n_samples), X]  # Adds a column of ones for the bias term

    # Initialize weights to zeros
    weights = np.zeros(n_features + 1)

    for epoch in range(int(n_iter)):
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        X_bias_shuffled = X_bias[indices]
        y_shuffled = y[indices]

        for i in range(0, n_samples, batch_size):
            X_batch = X_bias_shuffled[i:i + batch_size]
            y_batch = y_shuffled[i:i + batch_size]
            
            # Initialize gradients to zero
            gradient = np.zeros_like(weights)
            
            # Compute the gradient over the mini-batch
            for j in range(X_batch.shape[0]):
                # Prediction
                prediction = np.dot(X_batch[j], weights)
                error = y_batch[j] - prediction
                
                # Update gradient
                gradient += -2 * X_batch[j] * error

            # Update the weights
            weights -= learning_rate * gradient / batch_size

        # Check for convergence (if gradient is small enough)
        if np.linalg.norm(gradient) < epsilon:
            print(f"Converged after {epoch + 1} epochs")
            break

    return weights

# Hyperparameters
learning_rate = 0.01
n_iter = 1e4  # Number of epochs
epsilon = 1e-6  # Convergence threshold
batch_size = 16  # Mini-batch size

# Run SGD manually
weights_sgd = mini_batch_sgd(X_train, y_train, learning_rate, n_iter, epsilon=epsilon, batch_size=batch_size)

# Print the learned weights
print("Weights learned using SGD:\n", weights_sgd[1:])

Weights learned using SGD:
 [-11.00262763   5.00418417   0.63335422   2.4533718  -10.31607188
  17.4982821    1.02933367 -17.78123565   7.33294589  -6.42777006
  -8.80854031   3.0300865  -20.5191458 ]


In [4]:
num_samples = X_train.shape[0]
P = np.ones((num_samples, num_samples)) / num_samples # Equal probability to move to any state

alpha = 0.01  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e5  # Number of iterations
epsilon = 1e-9

td_sgd = TD_SGD(
    n_iter=num_iterations,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
    random_state=seed,
)

td_sgd.fit(X_train, y_train)

w_hat_house = td_sgd.weights
bias_house = td_sgd.bias

print("Weights learned using TD SGD:\n", w_hat_house)

Weights learned using TD SGD:
 [-10.69778107   4.95533416   0.47354431   2.06862183 -10.09323432
  18.0246395    1.23837445 -17.16714062   7.01245493  -6.71658602
  -8.93571416   2.9863624  -20.88360889]


In [6]:
alpha = 0.05  # Learning rate
gamma = 0   # Discount factor
num_iterations = 1e5  # Number of iterations
epsilon = 1e-9

td_adam = TD_Adam(
    n_iter=num_iterations,
    P=P,
    link=lambda x : x,
    inv_link=lambda x : x,
    gamma=gamma,
    alpha=alpha,
    epsilon=epsilon,
    random_state=seed,
)

td_adam.fit(X_train, y_train)

print("Weights learned using TD Adam:\n", td_adam.weights)

Weights learned using TD Adam:
 [-10.737863     4.6121483    0.31280735   1.3354009   -9.944276
  17.693153     1.3588558  -16.592127     6.8893633   -6.770194
  -9.154123     2.9209814  -20.840343  ]


In [15]:
pred_TD_sgd = td_sgd.predict(X_test)
pred_TD_adam = td_adam.predict(X_test)
pred_L2 = reg.predict(X_test)
pred_sgd = np.dot(X_test, weights_sgd[1:]) + weights_sgd[0]

rmse_TD_sgd = td_sgd.rmse(X_test, y_test)
rmse_TD_adam = td_adam.rmse(X_test, y_test)
rmse_L2 = np.sqrt(mean_squared_error(y_test, pred_L2))
rmse_sgd = np.sqrt(mean_squared_error(y_test, pred_sgd))

print("RMSE on the test set using TD SGD:", rmse_TD_sgd)
print("RMSE on the test set using TD Adam:", rmse_TD_adam)
print("RMSE on the test set using L2 Regression:", rmse_L2)
print("RMSE on the test set using SGD:", rmse_sgd)
print("---------------")
print("Norm of difference in weights for L2 and TD SGD:", np.linalg.norm(weights - td_sgd.weights, 2))
print("Norm of difference in weights for sgd and TD SGD:", np.linalg.norm(weights_sgd[1:] - td_sgd.weights, 2))
print("Norm of difference in weights for L2 and TD Adam:", np.linalg.norm(weights - td_adam.weights, 2))
print("Norm of difference in weights for sgd and TD Adam:", np.linalg.norm(weights_sgd[1:] - td_adam.weights, 2))


RMSE on the test set using TD SGD: 4.1246346540847725
RMSE on the test set using TD Adam: 4.245506682158816
RMSE on the test set using L2 Regression: 4.154652986817322
RMSE on the test set using SGD: 4.153448927184146
---------------
Norm of difference in weights for L2 and TD SGD: 1.1506830507130055
Norm of difference in weights for sgd and TD SGD: 1.1633814460756713
Norm of difference in weights for L2 and TD Adam: 2.382252727248095
Norm of difference in weights for sgd and TD Adam: 2.3737354229007175
